# Request VC from Authority

This notebook is used in combination with notebook `01_issue_VC_city.ipynb` (see Authority agent). Break points indicate when to switch notebooks.

#### Imports

In [1]:
%%javascript
document.title='City Agent'
%autoawait 
from aries_cloudcontroller import AriesAgentController
import asyncio
import os
from pprintpp import pprint
from termcolor import colored

IPython autoawait is `on`, and set to use `asyncio`


#### Initialize agent controller of City

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://city-agent:3021 and an api key of adminApiKey


## 1 – Init Webhook Server and register Event Listeners for it
#### 1.1 – Start webhook server
Start a webhook server to be able to communicate with other agents

In [3]:
# Setup
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.2 – Define listeners
Define listeners that are triggered when something happens on the webhook server

In [4]:
# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))

        
## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        print(credential)
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
        
        
def messages_handler(payload):
    connection_id = payload["connection_id"]
    agent_controller.messaging.send_message(connection_id, "This is a response from Bob")
    print("Handle message", connection_id)
    pprint(payload)

In [5]:
# Init listener list 
listeners = []

# Add listeners defined in previous cell
connection_listener = {"handler": connections_handler, "topic": "connections"}
listeners.append(connection_listener)

holder_listener = {"topic": "issue_credential", "handler": holder_handler}
listeners.append(holder_listener)

message_listener = {"handler": messages_handler, "topic": "basicmessages"}
listeners.append(message_listener)

#### 1.3 – Register listeners with `agent_controller`

In [6]:
#loop = asyncio.get_event_loop()
#loop.create_task(agent_controller.listen_webhooks())

agent_controller.register_listeners(listeners)

## 2 – Establish a connection with Authority agent
A connection with the credential issuer (i.e., the authority agent) must be establieshed before a VC can be received. In this scenario, the agent requests a connection with the Authority to be certified as an official city agency. Thus, the city agent sends an invitation to the Authority.

### 2.1 Create invitation to Authority agent

In [7]:
# Setup for connection with Authority agent
alias = None
auto_accept = False
public = "false" # Do not use public DID
multi_use = "false" # Invitation is only for one invitee

# Await response
invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)

# Keep connection_id to use it in other protocols (e.g., messaging)
connection_id = invitation_response["connection_id"]

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  aba3a43b-ef9a-4dde-9b78-962e515bd5f3
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation


### 2.2 – Send invitation with Authority
Sharing the invitation can be done via e.g., phone, zoom, E-Mail or QR-code. In this scenario, please copy and paste the invitation into `01_issue_VC_city.ipynb` of the Authority agent (see Step 2.1)

In [8]:
# Get and print invitation
invitation = invitation_response["invitation"]
print("Copy the following dict and past it into the Authority agency")
pprint(invitation)

Copy the following dict and past it into the Authority agency
{
    '@id': '21e83a12-1aee-4a14-968c-1841f80e259b',
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',
    'label': 'City',
    'recipientKeys': ['DbfJq8DBWcMWwgWM59Sph49wrxBooUyQoH5GK4AtwAN7'],
    'serviceEndpoint': 'https://727e9e2dfced.ngrok.io',
}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  aba3a43b-ef9a-4dde-9b78-962e515bd5f3
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
request


**BREAK POINT:** Please switch to agent `Authority`, open `01_issue_VC_city.ipynb`, and continue with Step 2

---

### 2.3 – Accept request response by Authority agent

In [9]:
# Accept request response of Authority
accept_request_response = await agent_controller.connections.accept_request(connection_id)

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  aba3a43b-ef9a-4dde-9b78-962e515bd5f3
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
response


### 2.4 – Sent trust ping to establish and activate connection

In [10]:
await agent_controller.messaging.trust_ping(connection_id, "send trust ping")

{'thread_id': '49f362b7-9f28-4e5c-b69f-a3a3deee537c'}

----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  aba3a43b-ef9a-4dde-9b78-962e515bd5f3
State :  active
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
Connection ID: aba3a43b-ef9a-4dde-9b78-962e515bd5f3 is now active.


## 3 – Request VC from `Authority` agent
### 3.1 – Message `Authority` to request a VC

In [11]:
basic_message = "Hello Authority agent"
await agent_controller.messaging.send_message(connection_id, basic_message)

{}

/tmp/ipykernel_44/3503404827.py:75: RuntimeWarning: coroutine 'MessagingController.send_message' was never awaited
  agent_controller.messaging.send_message(connection_id, "This is a response from Bob")


Handle message aba3a43b-ef9a-4dde-9b78-962e515bd5f3
{
    'connection_id': 'aba3a43b-ef9a-4dde-9b78-962e515bd5f3',
    'content': 'Hello City of Berlin',
    'message_id': '660942b2-b2c6-4634-80b0-c152bbe7891e',
    'sent_time': '2021-08-04 10:03:38.819141Z',
    'state': 'received',
}


**BREAK POINT:** Go to Step 3 in the `01_issue_VC_city.ipynb` notebook of the `Authority` agent.

---

### 3.1 – Provide `Authority` with the relevant information to issue a VC

In [12]:
basic_message = "I would like the get a VC proving that I am an official city agency in Berlin, DE."
await agent_controller.messaging.send_message(connection_id, basic_message)

{}


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : aba3a43b-ef9a-4dde-9b78-962e515bd5f3
Credential exchange ID : 3a20267f-d384-4d18-9570-814b066770ce
Agent Protocol Role :  holder
Protocol State :  offer_received

---------------------------------------------------

Handle Credential Webhook Payload
Credential Offer Recieved
The proposal dictionary is likely how you would understand and display a credential offer in your application

 {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/propose-credential', '@id': '952edec3-a398-4cb3-8000-e1bffd966204', 'schema_id': 'KxYXF6vfTXMoBNg2uGt79g:2:certify-city-agency:0.0.1', 'comment': 'Issuing VC that City is an agency', 'cred_def_id': 'KxYXF6vfTXMoBNg2uGt79g:3:CL:240881:default', 'credential_proposal': {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview', 'attributes': [{'name': 'city', 'value': 'Berlin'}, {'name': 'country', 'value': 'DE'}

**BREAK POINT:** Return to step 

---


### 3.2 – Request VC from `Authority`'s Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.

In [13]:
# Setup arguments for response
thread_id=None
state = "offer_received"
role = "prover"

records_response = await agent_controller.issuer.get_records(connection_id=connection_id,state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [14]:
# Send request for VC
await agent_controller.issuer.send_request_for_record(record_id)

{'thread_id': '0a493f65-6a25-441e-8bad-7c6bff696f02',
 'credential_offer': {'schema_id': 'KxYXF6vfTXMoBNg2uGt79g:2:certify-city-agency:0.0.1',
  'cred_def_id': 'KxYXF6vfTXMoBNg2uGt79g:3:CL:240881:default',
  'nonce': '465312571879895659738785',
  'key_correctness_proof': {'c': '77472694754129461971211245038695208553245659185091466704334720464424435716100',
   'xz_cap': '143246716461858300184964839223860599154922495729342244863497232521506640686669958669326209794368148140480756572795955018980020594878058535582549389579296340934269826061937818540600732391024420764620936869790797903027462462901063501499104726028177940131676115722258703401657072775191256122877107441385872855283517276028820639665806764083269759771942252634090489751943163298387261272623853163135238459289742524593929267803492006161679369488383794525429869526331802023029131770514895425373276402052174611387143016453826054882017465264036301650874050994217383484449797745626262471690068575683861515320109812602795813812576267403616


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : aba3a43b-ef9a-4dde-9b78-962e515bd5f3
Credential exchange ID : 3a20267f-d384-4d18-9570-814b066770ce
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : aba3a43b-ef9a-4dde-9b78-962e515bd5f3
Credential exchange ID : 3a20267f-d384-4d18-9570-814b066770ce
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential


### 3.3 – Store received VC in wallet

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [15]:
credential_id = "isCityAgency-VC"
store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)

ERROR:__name__:Error during POST /issue-credential/records/3a20267f-d384-4d18-9570-814b066770ce/store: 400, message='Error when storing credential in wallet: Error: Wallet item already exists. Caused by: Wallet item already exists with type: Indy::Credential, id: isCityAgency-VC. WalletItemAlreadyExists.', url=URL('http://city-agent:3021/issue-credential/records/3a20267f-d384-4d18-9570-814b066770ce/store')


ClientResponseError: 400, message='Error when storing credential in wallet: Error: Wallet item already exists. Caused by: Wallet item already exists with type: Indy::Credential, id: isCityAgency-VC. WalletItemAlreadyExists.', url=URL('http://city-agent:3021/issue-credential/records/3a20267f-d384-4d18-9570-814b066770ce/store')

ERROR:aiohttp.server:Error handling request
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/aiohttp/web_protocol.py", line 422, in _handle_request
    resp = await self._request_handler(request)
  File "/opt/conda/lib/python3.9/site-packages/aiohttp/web_app.py", line 499, in _handle
    resp = await handler(request)
  File "/opt/conda/lib/python3.9/site-packages/aries_cloudcontroller/aries_webhook_server.py", line 85, in _receive_webhook
    await self._handle_webhook(wallet_id, topic, payload)
  File "/opt/conda/lib/python3.9/site-packages/aries_cloudcontroller/aries_webhook_server.py", line 108, in _handle_webhook
    pub.sendMessage(pub_topic_path, payload=payload)
  File "/opt/conda/lib/python3.9/site-packages/pubsub/core/publisher.py", line 216, in sendMessage
    topicObj.publish(**msgData)
  File "/opt/conda/lib/python3.9/site-packages/pubsub/core/topicobj.py", line 452, in publish
    self.__sendMessage(msgData, topicObj, msgDataSubset)
  File 

## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [27]:
await agent_controller.terminate()